In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgbm
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import time
from sklearn.metrics import f1_score

In [3]:
DATA_DIR = './data/'
train_values = pd.read_csv(DATA_DIR + 'train_values.csv', index_col='building_id')
train_labels = pd.read_csv(DATA_DIR + 'train_labels.csv', index_col='building_id')
test_values = pd.read_csv(DATA_DIR + 'test_values.csv', index_col='building_id')
submission_format = pd.read_csv(DATA_DIR + 'submission_format.csv', index_col='building_id')

In [4]:
numerical_columns = [ 'count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage', 'count_families'] #'geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id',
categorical_columns = list(set(train_values.columns)-set(numerical_columns))

In [5]:
train_values = train_values.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
test_values = test_values.reset_index(drop=True)

In [6]:
# Handle categorical features that only appear at train set, but not test set.
train_feature = set(train_values['geo_level_2_id'].unique())
test_feature = set(test_values['geo_level_2_id'].unique())
unseen_features = test_feature - train_feature
for feature in unseen_features:
    # get geo_level_1_id of data that geo_level_2_id == feature
    geo_level_1_id = test_values[test_values['geo_level_2_id'] == feature]['geo_level_1_id'].values[0]
    # get the most common geo_level_1_id in the train_values taht has the same geo_level_2_id
    most_common = train_values[train_values['geo_level_1_id'] == geo_level_1_id]['geo_level_2_id'].value_counts().index[0]
    # replace the unseen feature with the most common one
    test_values.loc[test_values['geo_level_2_id'] == feature, 'geo_level_2_id'] = most_common

train_feature = set(train_values['geo_level_3_id'].unique())
test_feature = set(test_values['geo_level_3_id'].unique())
unseen_features = test_feature - train_feature
for feature in unseen_features:
    geo_level_2_id = test_values[test_values['geo_level_3_id'] == feature]['geo_level_2_id'].values[0]
    most_common = train_values[train_values['geo_level_2_id']==geo_level_2_id]['geo_level_3_id'].value_counts().index[0]
    test_values.loc[test_values['geo_level_3_id'] == feature, 'geo_level_3_id'] = most_common

In [7]:
#train_values = pd.get_dummies(train_values)# one-Hot Encoding
#test_values = pd.get_dummies(test_values) # one-Hot Encoding

categorical_index = [train_values.columns.get_loc(col) for col in categorical_columns]
for col in categorical_columns:
    train_values[col] = pd.Categorical(train_values[col])
    test_values[col] = pd.Categorical(test_values[col])


In [9]:
import optuna  # pip install optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback

def f1_micro(labels, preds):
    preds = preds.reshape(3, -1).T
    preds = preds.argmax(axis = 1)
    #preds = preds.reshape(-1,3).argmax(axis=1)
    is_higher_better = True
    return "f1_micro", f1_score(labels, preds, average='micro'), is_higher_better

def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 32, 1024, step=20),
        "max_depth": trial.suggest_int("max_depth", 5, 10),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 100, step=10),
        #"lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        #"lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        #"min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        #"bagging_fraction": trial.suggest_float(
        #    "bagging_fraction", 0.2, 0.95, step=0.1
        #),
        #"bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        #"feature_fraction": trial.suggest_float(
        #    "feature_fraction", 0.2, 0.95, step=0.1
        #),
    }

    N_SPLITS = 5
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = np.empty(N_SPLITS)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        print("=" * 12 + f"Training fold {idx+1}" + 12 * "=")
        start = time.time()

        X_train, X_val = train_values.iloc[train_idx], train_values.iloc[test_idx]
        y_train, y_val = train_labels.iloc[train_idx], train_labels.iloc[test_idx]    

        lgbm_clf = lgbm.LGBMClassifier(objective='multiclass', **param_grid)

        lgbm_clf.fit(
            X_train, y_train,
            eval_set=[(X_train, y_train), (X_val, y_val)],
            categorical_feature=categorical_index,
            early_stopping_rounds=100,
            eval_metric=f1_micro,
            verbose=False,
            callbacks=[
                LightGBMPruningCallback(trial, 'f1_micro', valid_name="valid_1")
            ],
        )

        preds = lgbm_clf.predict(X_val)
        score = f1_score(y_val, preds, average='micro')
        scores[idx] = score
        runtime = time.time() - start
        print(f"Fold {idx+1} finished with score: {score:.5f} in {runtime:.2f} seconds.\n")

    return np.mean(scores)

In [11]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, train_values, train_labels)
study.optimize(func, n_trials=20)

[I 2023-04-17 00:04:20,802] A new study created in memory with name: LGBM Classifier
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [32, 1024] and step=20, but the range is not divisible by `step`. It will be replaced by [32, 1012].
  warnings.warn(
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preproce

============Training fold 1============


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


Fold 1 finished with score: 0.74778 in 32.90 seconds.

============Training fold 2============
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

...

/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
[I 2023-04-17 00:22:24,775] Trial 17 pruned. Trial was pruned at iteration 0.
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [32, 1024] and step=20, but the range is not divisible by `step`. It will be replaced by [32, 1012].
  warnings.warn(
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

============Training fold 1============


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
[I 2023-04-17 00:22:25,668] Trial 18 pruned. Trial was pruned at iteration 0.
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [32, 1024] and step=20, but the range is not divisible by `step`. It will be replaced by [32, 1012].
  warnings.warn(
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

============Training fold 1============


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
[I 2023-04-17 00:22:26,054] Trial 19 pruned. Trial was pruned at iteration 0.


In [12]:
print(f"\tBest value (f1 score): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (f1 score): 0.75001
	Best params:
		n_estimators: 10000
		learning_rate: 0.10534606024975147
		num_leaves: 572
		max_depth: 5
		min_data_in_leaf: 30


In [8]:


params = {'n_estimators': 10000,
		'learning_rate': 0.10534606024975147,
		'num_leaves': 572,
		'max_depth': 5,
		'min_data_in_leaf': 30}

def f1_micro(labels, preds):
    preds = preds.reshape(3, -1).T
    preds = preds.argmax(axis = 1)
    #preds = preds.reshape(-1,3).argmax(axis=1)
    is_higher_better = True
    return "f1_micro", f1_score(labels, preds, average='micro'), is_higher_better


N_SPLITS = 5
strat_kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
scores = np.empty(N_SPLITS)
train_prediction = np.zeros((train_values.shape[0],3))
test_prediction = np.zeros((test_values.shape[0],3))
for idx, (train_idx, test_idx) in enumerate(strat_kf.split(train_values, train_labels)):
    print("=" * 12 + f"Training fold {idx}" + 12 * "=")
    start = time.time()

    X_train, X_val = train_values.iloc[train_idx], train_values.iloc[test_idx]
    y_train, y_val = train_labels.iloc[train_idx], train_labels.iloc[test_idx]    

    lgbm_clf = lgbm.LGBMClassifier(objective='multiclass', **params)

    lgbm_clf.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        categorical_feature=categorical_index,
        early_stopping_rounds=100,
        eval_metric=f1_micro,
        verbose=False,
    )

    preds = lgbm_clf.predict(X_val)
    score = f1_score(y_val, preds, average='micro')
    scores[idx] = score

    y_pred = lgbm_clf.predict_proba(X_val)
    train_prediction[test_idx, :] = y_pred

    y_pred = lgbm_clf.predict_proba(test_values)
    test_prediction += y_pred

    runtime = time.time() - start
    print(f"Fold {idx+1} finished with score: {score:.5f} in {runtime:.2f} seconds.\n")

test_prediction /= N_SPLITS
print('CV mean: {:.4f}, CV std: {:.4f}'.format(np.mean(scores), np.std(scores)))

============Training fold 0============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 1 finished with score: 0.74826 in 973.89 seconds.

============Training fold 1============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 2 finished with score: 0.75017 in 958.53 seconds.

============Training fold 2============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


...

Fold 3 finished with score: 0.75190 in 917.18 seconds.

============Training fold 3============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 4 finished with score: 0.75048 in 1032.33 seconds.

============Training fold 4============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 5 finished with score: 0.74925 in 1040.74 seconds.

CV mean: 0.7500, CV std: 0.0012


In [ ]:
prediction_labels = np.argmax(test_prediction, axis=1)+1
submission = pd.DataFrame(data=prediction_labels,
                             columns=submission_format.columns,
                             index=submission_format.index)
submission.to_csv('lightgbm_submission.csv')
submission.value_counts()

damage_grade
2               55499
3               24533
1                6836
dtype: int64

In [ ]:
# Pseudo-labeling

threshold = 0.8

test_X_pseudo_1 = test_values.iloc[np.where(test_prediction[:,0] > threshold)]
test_y_pseudo_1 = [1]*test_X_pseudo_1.shape[0]
test_X_pseudo_2 = test_values.iloc[np.where(test_prediction[:,1] > threshold)]
test_y_pseudo_2 = [2]*test_X_pseudo_2.shape[0]
test_X_pseudo_3 = test_values.iloc[np.where(test_prediction[:,2] > threshold)]
test_y_pseudo_3 = [3]*test_X_pseudo_3.shape[0]

test_pseudo = pd.concat([test_X_pseudo_1, test_X_pseudo_2, test_X_pseudo_3]).reset_index(drop = True)
test_pseudo['damage_grade'] = pd.Series(test_y_pseudo_1+test_y_pseudo_2+test_y_pseudo_3)
test_pseudo_y = pd.DataFrame(test_pseudo['damage_grade'])
test_pseudo_X = test_pseudo.drop('damage_grade', axis = 1)
#test_pseudo_sample = test_pseudo.sample(int(0.2*test_values.shape[0]))

train_X_pseudo = pd.concat([train_values, test_pseudo_X],ignore_index=True)
train_y_pseudo = pd.concat([train_labels, test_pseudo_y],ignore_index=True)

categorical_index = [train_values.columns.get_loc(col) for col in categorical_columns]
for col in categorical_columns:
    train_X_pseudo[col] = pd.Categorical(train_X_pseudo[col])

N_SPLITS = 5
strat_kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
scores = np.empty(N_SPLITS)
train_prediction = np.zeros((train_X_pseudo.shape[0],3))
test_prediction = np.zeros((test_values.shape[0],3))
for idx, (train_idx, test_idx) in enumerate(strat_kf.split(train_X_pseudo, train_y_pseudo)):
    print("=" * 12 + f"Training fold {idx}" + 12 * "=")
    start = time.time()

    X_train, X_val = train_X_pseudo.iloc[train_idx], train_X_pseudo.iloc[test_idx]
    y_train, y_val = train_y_pseudo.iloc[train_idx], train_y_pseudo.iloc[test_idx]    

    lgbm_clf = lgbm.LGBMClassifier(objective='multiclass', **params)

    lgbm_clf.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train), (X_val, y_val)],
        categorical_feature=categorical_index,
        early_stopping_rounds=100,
        eval_metric=f1_micro,
        verbose=False,
    )

    preds = lgbm_clf.predict(X_val)
    score = f1_score(y_val, preds, average='micro')
    scores[idx] = score

    y_pred = lgbm_clf.predict_proba(X_val)
    train_prediction[test_idx, :] = y_pred

    y_pred = lgbm_clf.predict_proba(test_values)
    test_prediction += y_pred

    runtime = time.time() - start
    print(f"Fold {idx+1} finished with score: {score:.5f} in {runtime:.2f} seconds.\n")

test_prediction /= N_SPLITS
print('CV mean: {:.4f}, CV std: {:.4f}'.format(np.mean(scores), np.std(scores)))


============Training fold 0============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 1 finished with score: 0.78605 in 80.24 seconds.

============Training fold 1============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 2 finished with score: 0.78322 in 91.72 seconds.

============Training fold 2============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


...

Fold 3 finished with score: 0.78400 in 100.75 seconds.

============Training fold 3============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 4 finished with score: 0.78428 in 95.99 seconds.

============Training fold 4============
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/dongwon/.conda/envs/ml/lib/python3.9/site-packages/lightgbm/basic.py:1702: UserWarning: Usin

Fold 5 finished with score: 0.77976 in 76.12 seconds.

CV mean: 0.7835, CV std: 0.0021


In [ ]:

prediction_labels = np.argmax(test_prediction, axis=1)+1
submission = pd.DataFrame(data=prediction_labels,
                             columns=submission_format.columns,
                             index=submission_format.index)
submission.to_csv('pseudo_lightgbm_submission.csv')
submission.value_counts()

damage_grade
2               55548
3               24492
1                6828
dtype: int64

In [9]:
test_prediction_df = pd.DataFrame(data=test_prediction,
                             columns=['1','2','3'],
                             index=submission_format.index)
test_prediction_df.to_csv('test_pseudo_lightgbm_prediction.csv')
test_prediction_df.shape

(86868, 3)

In [10]:
train_prediction_df = pd.DataFrame(data=train_prediction[:train_values.shape[0],:],
                             columns=['1','2','3'])
train_prediction_df.to_csv('train_pseudo_lightgbm_prediction.csv')
train_prediction_df.shape

(260601, 3)

In [71]:
train_values.shape

(260601, 38)

In [72]:
test_values.shape

(86868, 38)